In [2]:
from pyspark import SQLContext
sqlContext=SQLContext(sc)
df=sqlContext.read.format('csv')\
    .options(header='true',inferschema='true')\
    .load('s3://pgarias-bucket-cloud/311_Service_Requests_from_2015_to_Present.csv')


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
df.describe().show()

In [3]:
df2=df.drop('Unique Key','Agency Name','BBL','Location','Location Type', 'Cross Street 1', 'Cross Street 2', 'Intersection Street 1',
             'Intersection Street 2','Landmark', 'Due Date', 'Vehicle Type', 'Taxi Company Borough',
             'Taxi Pick Up Location', 'Bridge Highway Name', 'Bridge Highway Direction', 'Road Ramp',
             'Bridge Highway Segment')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
from pyspark.sql.functions import col,countDistinct
df2.agg(*(countDistinct(col(c)).alias(c) for c in df2.columns)).show()

In [4]:
df3=df2.drop('Descriptor','Incident Zip','Incident Address',
             'Street Name','City','Resolution Description',
            'Resolution Action Updated Date','X Coordinate (State Plane)',
            'Y Coordinate (State Plane)','Park Facility Name',
            'Latitude','Longitude')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
df4=df3.filter(df3.Status=='Closed')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
df5=df4.dropna()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
from pyspark.sql.functions import udf, col
from pyspark.sql.types import FloatType,IntegerType,StringType
def month_to_season(date):
    date=str(date)
    monthint=int(date[0:2])
    if monthint>=3 and monthint<=5:
        return 'Spring'
    if monthint>=6 and monthint<=8:
        return 'Summer'
    if monthint>=9 and monthint<=11:
        return 'Fall'
    if monthint==12 or monthint==1 or monthint==2:
        return 'Winter'
udfmonth_to_season=udf(month_to_season,StringType())
df6=df5.withColumn("Season",udfmonth_to_season("Created Date"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
from pyspark.sql.functions import from_unixtime,unix_timestamp
from pyspark.sql.types import DateType
timeFmt = 'MM/dd/yyyy hh:mm:ss a'
timeDiff = (unix_timestamp('Closed Date', format=timeFmt)
            - unix_timestamp('Created Date', format=timeFmt))
df7 = df6.withColumn("Duration", timeDiff)
def int_to_float(x):
    return float(x)
udfint_to_float = udf(int_to_float, FloatType())
df8 = df7.withColumn("Duration_float", udfint_to_float("Duration"))

In [13]:
df8.groupBy('Season').sum().collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(Season='Spring', sum(Duration)=2131007277207, sum(Duration_float)=2131007276822.0), Row(Season='Summer', sum(Duration)=2715251668456, sum(Duration_float)=2715251668433.0), Row(Season='Fall', sum(Duration)=2429944692993, sum(Duration_float)=2429944692861.0), Row(Season='Winter', sum(Duration)=2461082986694, sum(Duration_float)=2461082986216.0)]

In [9]:
df9=df8.drop('Created Date','Closed Date','Status')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
df10=df9.select('Duration','Duration_float','Agency','Complaint Type',
                'Address Type','Facility Type','Community Board','Borough',
                'Open Data Channel Type','Park Borough','Season')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
df10.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------+------+--------------------+------------+-------------+---------------+---------+----------------------+------------+------+
|Duration|Duration_float|Agency|      Complaint Type|Address Type|Facility Type|Community Board|  Borough|Open Data Channel Type|Park Borough|Season|
+--------+--------------+------+--------------------+------------+-------------+---------------+---------+----------------------+------------+------+
|   10020|       10020.0|  NYPD|    Blocked Driveway|     ADDRESS|     Precinct|      10 QUEENS|   QUEENS|                 PHONE|      QUEENS|Winter|
|    9653|        9653.0|  NYPD|    Blocked Driveway|     ADDRESS|     Precinct|       12 BRONX|    BRONX|                 PHONE|       BRONX|Winter|
|    1143|        1143.0|  NYPD|Noise - Street/Si...|     ADDRESS|     Precinct|   09 MANHATTAN|MANHATTAN|                ONLINE|   MANHATTAN|Winter|
|    8434|        8434.0|  NYPD|     Noise - Vehicle|     ADDRESS|     Precinct|       07 BRONX|    

In [11]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer
indexing=[StringIndexer(inputCol=col,outputCol=col+"indexed").fit(df10) 
          for col in list(set(df10.columns)-set(['Duration_float']))]
pipeline=Pipeline(stages=indexing)
df10_i=pipeline.fit(df10).transform(df10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
df10_i.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------+------+--------------------+------------+-------------+---------------+---------+----------------------+------------+------+---------------------+-------------+--------------+-------------------+--------------------+-------------+-----------------------------+---------------+-------------------+----------------------+
|Duration|Duration_float|Agency|      Complaint Type|Address Type|Facility Type|Community Board|  Borough|Open Data Channel Type|Park Borough|Season|Complaint Typeindexed|Agencyindexed|Boroughindexed|Park Boroughindexed|Facility Typeindexed|Seasonindexed|Open Data Channel Typeindexed|Durationindexed|Address Typeindexed|Community Boardindexed|
+--------+--------------+------+--------------------+------------+-------------+---------------+---------+----------------------+------------+------+---------------------+-------------+--------------+-------------------+--------------------+-------------+-----------------------------+---------------+---------

In [12]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols = ['Agencyindexed', 'Boroughindexed','Complaint Typeindexed', 
                                               'Park Boroughindexed', 'Facility Typeindexed', 'Seasonindexed', 
                                               'Open Data Channel Typeindexed', 'Address Typeindexed', 
                                               'Community Boardindexed'], outputCol = 'features')
vdf10_i = vectorAssembler.transform(df10_i)
vdf10_i = vdf10_i.select(['features', 'Duration_float'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
(trainingData, testData) = vdf10_i.randomSplit([0.7, 0.3])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
from pyspark.ml.regression import DecisionTreeRegressor
regressor=DecisionTreeRegressor(featuresCol='features',labelCol='Duration_float',maxBins=250)
model=regressor.fit(trainingData)
model.featureImportances

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparseVector(9, {0: 0.0029, 1: 0.0005, 2: 0.9776, 5: 0.0003, 6: 0.0077, 7: 0.0009, 8: 0.01})

In [16]:
from pyspark.ml.evaluation import RegressionEvaluator
model_predictions=model.transform(testData)
model_evaluator=RegressionEvaluator(labelCol='Duration_float',predictionCol='prediction',
                                    metricName='rmse')
rmse=model_evaluator.evaluate(model_predictions)
print(rmse)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

10964014.732892832